## Scenario 3: Multiple data scientists working on multiple ML models

MLflow setup:
* Tracking server: yes, remote server (EC2).
* Backend store: postgresql database.
* Artifacts store: s3 bucket.

The experiments can be explored by accessing the remote server.

The exampe uses AWS to host a remote server. In order to run the example you'll need an AWS account. Follow the steps described in the file `mlflow_on_aws.md` to create a new AWS account and launch the tracking server. 

In [20]:
import mlflow
import os
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
import pickle

import pandas as pd

from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import make_pipeline


In [21]:
os.environ["AWS_PROFILE"] = "ArunG" # fill in with your AWS profile. More info: https://docs.aws.amazon.com/sdk-for-java/latest/developer-guide/setup.html#setup-credentials

TRACKING_SERVER_HOST = "http://127.0.0.1:5000"
mlflow.set_tracking_uri(TRACKING_SERVER_HOST)
mlflow.set_experiment("green-taxi-duration")

#TRACKING_SERVER_HOST = "ec2-13-60-29-159.eu-north-1.compute.amazonaws.com" # fill in with the public DNS of the EC2 instance
#mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:5000")

<Experiment: artifact_location='s3://mlflow-artifacts-gansi/1', creation_time=1718859304481, experiment_id='1', last_update_time=1718859304481, lifecycle_stage='active', name='green-taxi-duration', tags={}>

In [22]:
!mlflow --version

mlflow, version 2.14.0


In [23]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'", TRACKING_SERVER_HOST)

tracking URI: 'http://127.0.0.1:5000' http://127.0.0.1:5000


In [24]:
# mlflow.list_experiments()

In [25]:
!pip3 install boto3

In [26]:
def read_dataframe(filename: str):
    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    return df


def prepare_dictionaries(df: pd.DataFrame):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    dicts = df[categorical + numerical].to_dict(orient='records')
    return dicts

In [27]:
df_train = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet')

target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

dict_train = prepare_dictionaries(df_train)
dict_val = prepare_dictionaries(df_val)

In [28]:
# with mlflow.start_run():

#     X, y = load_iris(return_X_y=True)

#     params = {"C": 0.1, "random_state": 42}
#     mlflow.log_params(params)

#     lr = LogisticRegression(**params).fit(X, y)
#     y_pred = lr.predict(X)
#     mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

#     mlflow.sklearn.log_model(lr, artifact_path="models")
#     print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

In [29]:
# mlflow.list_experiments()

In [30]:
with mlflow.start_run():
   params = dict(max_depth=20, n_estimators=100, min_samples_leaf=10, random_state=0)
   mlflow.log_params(params)

   dv = DictVectorizer()
   model = RandomForestRegressor(**params, n_jobs=-1)

   X_train = dv.fit_transform(dict_train)
   model.fit(X_train,y_train)
    
   X_val = dv.transform(dict_val)
   y_pred = model.predict(X_val)

   rmse = mean_squared_error(y_pred, y_val, squared=False)
   print(params, rmse)
   mlflow.log_metric('rmse', rmse)

   mlflow.sklearn.log_model(model, artifact_path="model")
    
   with open('dict_vectorizer.bin','wb') as f_out:
      pickle.dump(dv,f_out)
    
   mlflow.log_artifact('dict_vectorizer.bin')

c:\Users\ArunGanesan\mlops-zoomcamp-garun\exp-tracking\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


{'max_depth': 20, 'n_estimators': 100, 'min_samples_leaf': 10, 'random_state': 0} 6.7558229919200725


### Interacting with the model registry

In [31]:
from mlflow.tracking import MlflowClient


client = MlflowClient(f"http://{TRACKING_SERVER_HOST}:5000")

In [32]:
# client.list_registered_models()

In [33]:
# run_id = client.list_run_infos(experiment_id='1')[0].run_id
# mlflow.register_model(
#     model_uri=f"runs:/{run_id}/models",
#     name='iris-classifier'
# )

In [34]:
MLFLOW_TRACKING_URI = 'http://127.0.0.1:5000'
RUN_ID = '32e47515143a4fdc80b33cf53d9e58ff'

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [35]:
path = client.download_artifacts(run_id=RUN_ID, path='dict_vectorizer.bin')

In [36]:
print(path)

C:\Users\ARUNGA~1\AppData\Local\Temp\tmp6lphvrr_\dict_vectorizer.bin
